In [1]:
import biotite.structure as struc
from biotite.structure import AtomArray
from biotite.structure.io import pdbx
from biotite.structure.io.pdb import PDBFile
from io import StringIO
import pandas as pd


In [2]:
# Read XYZ coordinates from all-atom file

In [17]:
fn = '../output/v311_2025_01-01_2025-10-01/v311_2025-01-01_2025-10-01_allatom.csv'
target_id = '8K0Y_B'

allatom = pd.read_csv(fn)
allatom = allatom.query('group_id == @target_id')
# Get first model
allatom = allatom.set_index(['ID', 'group_id', 'resname', 'resid'])
allatom = allatom[allatom.columns[allatom.columns.str.endswith('_1')]]
allatom.columns = allatom.columns.str.rstrip('_1')
#allatom = allatom.reset_index()
# Convert to biotite AtomArray
# Make it long array with separate xyz and atom info columns (derived from initial column names)
allatom.columns = allatom.columns.str.split('_', expand=True)
allatom.columns.names = ['atom_name', 'coord']
allatom = allatom.stack(level='atom_name').reset_index()
allatom['element'] = allatom['atom_name'].str[0]
allatom['chain_id'] = 'A' # We have single chain right now, make it compatible with PDB format

/tmp/ipykernel_82762/350478575.py:15: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  allatom = allatom.stack(level='atom_name').reset_index()


In [4]:
# Convert to AtomArray
aa = AtomArray(len(allatom))
aa.coord = allatom[['x', 'y', 'z']].to_numpy().astype(float)
aa.element = allatom['element'].to_numpy().astype("U2")
aa.atom_name = allatom['atom_name'].to_numpy().astype('U6')
aa.res_id = allatom['resid'].to_numpy().astype(int)
aa.res_name = allatom['resname'].to_numpy().astype('U5')
aa.chain_id = allatom['chain_id'].to_numpy().astype('U4')

In [15]:
# Write to PDB file
output_file = f'{target_id}.pdb'

pdb_file = PDBFile()
pdb_file.set_structure(aa)
pdb_file.write(output_file)

In [16]:
# Display
import nglview
nglview.show_file(output_file)

NGLWidget()

NGLWidget()